# Benchmarks

In [1]:
# the notebook imports
import matplotlib.pyplot as plt
import numpy as np
# this is the convenience function
from autokoopman import auto_koopman
# for a complete example, let's create an example dataset using an included benchmark system
from autokoopman.benchmark import bio2, fhn, lalo20, prde20, robe21, spring, pendulum
from glop import Glop
import random
import copy

from sklearn.metrics import mean_squared_error
import statistics
import os
import csv
import time

In [2]:
#benches = [bio2.Bio2(), fhn.FitzHughNagumo(), lalo20.LaubLoomis(), pendulum.PendulumWithInput(beta=0.05), prde20.ProdDestr(), robe21.RobBench(), spring.Spring()]
# TODO: implement support for constraints on init set, maybe use glop solver
benches = [spring.Spring()]

In [3]:
def get_training_data(bench, param_dict):
    init_states = get_init_states(bench, param_dict["train_size"])
    if bench._input_vars:
        params = np.random.rand(param_dict["train_size"], 3) * 2 - 1
        steps = [make_input_step(*p, bench.teval) for p in params]
        training_data = bench.solve_ivps(initial_states=init_states, inputs=steps, teval=bench.teval)
    else:
        training_data = bench.solve_ivps(initial_states=init_states,tspan=[0.0, 10.0], 
                                         sampling_period=param_dict["samp_period"])
        
    return training_data

In [4]:
def get_init_states(bench, size, iterat=0):
    if hasattr(bench, 'init_constrs'):
        init_states = []
        for i in range(size):
            init_state_dict = glop_init_states(bench, i+iterat)
            init_state = []
            for name in bench.names:
                init_state.append(init_state_dict[name])
            init_states.append(init_state)
        init_states = np.array(init_states)  
    else:
        init_states = np.random.uniform(low=bench.init_set_low, 
                    high=bench.init_set_high, size=(param_dict["train_size"], len(bench.names)))
        
    print(init_states)
    return init_states

In [5]:
def glop_init_states(bench, seed):    
    constrs = []
    for constr in bench.init_constrs:
        constrs.append(constr)
    for i, (name, init_low, init_high) in enumerate(zip(bench.names, bench.init_set_low, bench.init_set_high)):
        low_constr = f"{name} >= {init_low}"
        high_constr = f"{name} <= {init_high}"
        constrs.extend([low_constr, high_constr])
        
    glop = Glop(bench.names, constrs)
    pop_item = random.randrange(len(bench.names))
    names, init_set_low, init_set_high = copy.deepcopy(bench.names), copy.deepcopy(bench.init_set_low), copy.deepcopy(bench.init_set_high)
    names.pop(pop_item)
    init_set_low.pop(pop_item)
    init_set_high.pop(pop_item)
    for i, (name, init_low, init_high) in enumerate(zip(names, init_set_low, init_set_high)):
        glop.add_tend_value_obj_fn(name, [init_low, init_high], seed+i)
    
    glop.minimize()

    sol_dict = glop.get_all_sols()    
    return sol_dict

In [6]:
def get_trajectories(bench, iv):
    # get the model from the experiment results
    model = experiment_results['tuned_model']

    if bench._input_vars:
        test_inp = np.sin(np.linspace(0, 10, 200))

        # simulate using the learned model
        trajectory = model.solve_ivp(
            initial_state=iv,
            inputs=test_inp,
            teval=bench.teval,
        )
        # simulate the ground truth for comparison
        true_trajectory = bench.solve_ivp(
            initial_state=iv,
            inputs=test_inp,
            teval=bench.teval,
        )
        
    else:
        # simulate using the learned model
        trajectory = model.solve_ivp(
            initial_state=iv,
            tspan=(0.0, 10.0),
            sampling_period=0.1
        )
        # simulate the ground truth for comparison
        true_trajectory = bench.solve_ivp(
            initial_state=iv,
            tspan=(0.0, 10.0),
            sampling_period=0.1
        )
    
    return trajectory, true_trajectory

In [7]:
def test_trajectories(bench, num_tests):
    mses = []
    mses_dim = [[] for x in range(len(bench.names))]
    for j in range(num_tests):
        iv = get_init_states(bench, 1, iterat=j+1000)[0]
        trajectory, true_trajectory = get_trajectories(bench, iv)
        mse = mean_squared_error(trajectory.states.T, true_trajectory.states.T)
        mses.append(mse)
        
        for traj_dim, (trajectory, true_trajectory) in enumerate(zip(trajectory.states.T, true_trajectory.states.T)):
            mse = mean_squared_error(trajectory, true_trajectory)
            mses_dim[traj_dim].append(mse)
            
    return mses, mses_dim

In [8]:
def make_input_step(duty, on_amplitude, off_amplitude, teval):
    """produce a step response input signal for the pendulum"""
    length = len(teval)
    inp = np.zeros((length,))
    phase_idx = int(length * duty)
    inp[:phase_idx] = on_amplitude
    inp[phase_idx:] = off_amplitude
    return inp

In [9]:
def store_data(bench_name, dim, param_name, param_values, train_times, all_mses, all_mses_dim):
    if not os.path.exists('data'):
        os.makedirs('data')
        
    with open(f'data/{bench_name}', 'a') as f:
        writer = csv.writer(f)
        row = [param_name, "train_time", "Avg mse"]
        for i in range(dim):
              row.append(f'Avg mse dim {i+1}')
        writer.writerow(row)
              
        for param_value, train_time, mse, mses_dim in zip(param_values, train_times, all_mses, all_mses_dim):
              row = [param_value, train_time, mse]
              for mses in mses_dim:
                  row.append(mses)
              writer.writerow(row)
        writer.writerow([])

In [10]:
def plot(trajectory, true_trajectory):
    plt.figure(figsize=(10, 6))
    # plot the results
    plt.plot(trajectory.states.T[0], trajectory.states.T[1],label='Trajectory Prediction')
    plt.plot(true_trajectory.states.T[0], true_trajectory.states.T[1],label='Ground Truth')

    plt.xlabel("$x_1$")
    plt.ylabel("$x_2$")
    plt.grid()
    plt.legend()
    plt.title("Bio2 Test Trajectory Plot")
    plt.show()

In [11]:
all_param_values = {
    "train_size":[5, 10, 25, 50, 75, 100, 125, 150, 175, 200],
    "samp_period":[0.01, 0.025, 0.05, 0.08, 0.1, 0.2, 0.25, 0.5, 1, 2],
    "obs_type":["rff","quadratic", "id"],
    "opt":["grid", "monte-carlo"],
    "n_obs":[5, 10, 25, 50, 100, 200, 300, 400, 500, 1000],
    "grid_param_slices":[5, 10, 25, 50, 100],
    "n_splits":[2, 5, 10],
    "rank":[(1, 200, 40)]
}

In [ ]:
for bench in benches:
    for param, param_values in all_param_values.items():
        all_mses = []
        all_mses_dim = []
        times = []
        param_dict = {"train_size":10,"samp_period":0.1,"obs_type":"rff","opt":"grid","n_obs":200,
                      "grid_param_slices":5,"n_splits":5,"rank":(1, 200, 40)}
        for i, param_value in enumerate(param_values):
            np.random.seed(0)
            param_dict[param] = param_value
            start = time.time()
            # generate training data
            training_data = get_training_data(bench, param_dict)
            # learn model from data
            experiment_results = auto_koopman(
                training_data,          # list of trajectories
                sampling_period=0.1,    # sampling period of trajectory snapshots
                obs_type=param_dict["obs_type"],         # use Random Fourier Features Observables
                opt=param_dict["opt"],             # grid search to find best hyperparameters
                n_obs=param_dict["n_obs"],              # maximum number of observables to try
                max_opt_iter=200,       # maximum number of optimization iterations
                grid_param_slices=param_dict["grid_param_slices"],# for grid search, number of slices for each parameter
                n_splits=param_dict["n_splits"],             # k-folds validation for tuning, helps stabilize the scoring
                rank=param_dict["rank"]       # rank range (start, stop, step) DMD hyperparameter
            )
            end = time.time()
            times.append(round(end - start, 3))
            all_mses_dim.append([])
            
            try:
                mses, mses_dim = test_trajectories(bench, 10)
                print("The average mean square error is ", statistics.mean(mses))
                all_mses.append(statistics.mean(mses))
                for traj_dim, mses in enumerate(mses_dim):
                    all_mses_dim[i].append(statistics.mean(mses))
                    print(f"The average mean square error for dim {traj_dim+1} is", statistics.mean(mses))
            except ValueError:
                print("benchmark did not compute for this setting")

        store_data(bench.name, len(bench.names), param, param_values, times, all_mses, all_mses_dim)

[[1.20097627 0.50430379 0.         0.        ]
 [1.1984731  0.50291788 0.         0.        ]
 [1.20927326 0.49766883 0.         0.        ]
 [1.20136089 0.50851193 0.         0.        ]
 [1.19040437 0.5066524  0.         0.        ]]


  0%|                                                    | 0/25 [00:00<?, ?it/s]/Users/b6062805/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:2559: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real
  4%|█▊                                          | 1/25 [00:00<00:06,  3.55it/s]/Users/b6062805/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:2560: RuntimeWarning: overflow encountered in reduce
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))
/usr/local/anaconda3/lib/python3.8/site-packages/autokoopman/estimator/koopman.py:83: RuntimeWarning: overflow encountered in matmul
  return np.real(self._A @ obs.T).flatten()[: len(x)]
/usr/local/anaconda3/lib/python3.8/site-packages/autokoopman/core/observables.py:180: RuntimeWarning: invalid value encountered in matmul
  Z = s * np.cos(x.T @ w + u.T)
/usr/local/anaconda3/lib/python3.8/site-packages/autokoopman/core/observables.py:180: RuntimeWarning: invalid value encountered in cos
  Z = s

[[1.19503721 0.4908881  0.         0.        ]
 [1.19841246 0.5064563  0.         0.        ]
 [1.19320772 0.49526656 0.         0.        ]
 [1.20047892 0.49357205 0.         0.        ]
 [1.19218203 0.49329484 0.         0.        ]]
[[1.20645858 0.50885778 0.         0.        ]
 [1.19861158 0.49535312 0.         0.        ]
 [1.20285348 0.50308053 0.         0.        ]
 [1.19840723 0.5049049  0.         0.        ]
 [1.19865442 0.50472004 0.         0.        ]]
[[1.19826505 0.50923766 0.         0.        ]
 [1.19862194 0.50021717 0.         0.        ]
 [1.20656588 0.50967298 0.         0.        ]
 [1.19747129 0.49046002 0.         0.        ]
 [1.2092048  0.5033374  0.         0.        ]]
[[1.20922796 0.49784796 0.         0.        ]
 [1.20716361 0.50622212 0.         0.        ]
 [1.20165903 0.49835483 0.         0.        ]
 [1.20579213 0.50072714 0.         0.        ]
 [1.20295411 0.50974958 0.         0.        ]]
[[1.19396153 0.50373347 0.         0.        ]
 [1.20420

  0%|                                                    | 0/25 [00:00<?, ?it/s]/Users/b6062805/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:2559: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real
/Users/b6062805/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:2560: RuntimeWarning: overflow encountered in reduce
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))
  4%|█▊                                          | 1/25 [00:00<00:12,  1.89it/s]/usr/local/anaconda3/lib/python3.8/site-packages/autokoopman/estimator/koopman.py:83: RuntimeWarning: overflow encountered in matmul
  return np.real(self._A @ obs.T).flatten()[: len(x)]
/usr/local/anaconda3/lib/python3.8/site-packages/autokoopman/estimator/koopman.py:83: RuntimeWarning: invalid value encountered in matmul
  return np.real(self._A @ obs.T).flatten()[: len(x)]
/usr/local/anaconda3/lib/python3.8/site-packages/scipy/interpolate/_interpolate.py:641: RuntimeWarning: overflow encount

[[1.20295411 0.50974958 0.         0.        ]
 [1.19396153 0.50373347 0.         0.        ]
 [1.20420046 0.50671273 0.         0.        ]
 [1.19899135 0.50950926 0.         0.        ]
 [1.19179837 0.49740717 0.         0.        ]
 [1.20246135 0.50282226 0.         0.        ]
 [1.19765585 0.49826982 0.         0.        ]
 [1.20373955 0.49500108 0.         0.        ]
 [1.19460093 0.49862561 0.         0.        ]
 [1.19863816 0.50812236 0.         0.        ]]
[[1.19210015 0.49837405 0.         0.        ]
 [1.20973717 0.50096448 0.         0.        ]
 [1.20794952 0.49186335 0.         0.        ]
 [1.20388865 0.49806781 0.         0.        ]
 [1.20750956 0.49763324 0.         0.        ]
 [1.19258384 0.50744275 0.         0.        ]
 [1.20832507 0.50378642 0.         0.        ]
 [1.20089711 0.49443954 0.         0.        ]
 [1.19033667 0.50476172 0.         0.        ]
 [1.19418611 0.49054548 0.         0.        ]]
[[1.1900218  0.50306531 0.         0.        ]
 [1.1973919

...

  0%|                                                    | 0/50 [00:00<?, ?it/s]/Users/b6062805/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:2559: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real
/Users/b6062805/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:2560: RuntimeWarning: overflow encountered in reduce
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))
  2%|▉                                           | 1/50 [00:00<00:24,  1.97it/s]/usr/local/anaconda3/lib/python3.8/site-packages/autokoopman/estimator/koopman.py:83: RuntimeWarning: overflow encountered in matmul
  return np.real(self._A @ obs.T).flatten()[: len(x)]
/usr/local/anaconda3/lib/python3.8/site-packages/autokoopman/estimator/koopman.py:83: RuntimeWarning: invalid value encountered in matmul
  return np.real(self._A @ obs.T).flatten()[: len(x)]
/usr/local/anaconda3/lib/python3.8/site-packages/scipy/interpolate/_interpolate.py:641: RuntimeWarning: overflow encount

[[1.20828174 0.49272618 0.         0.        ]
 [1.19970439 0.49800002 0.         0.        ]
 [1.20589878 0.49014418 0.         0.        ]
 [1.19694799 0.50619855 0.         0.        ]
 [1.19259812 0.50029821 0.         0.        ]
 [1.20035206 0.50924794 0.         0.        ]
 [1.20282506 0.50611779 0.         0.        ]
 [1.19172711 0.49600907 0.         0.        ]
 [1.2030206  0.4941219  0.         0.        ]
 [1.2068558  0.4997199  0.         0.        ]]
[[1.20254469 0.50167031 0.         0.        ]
 [1.19979196 0.50347211 0.         0.        ]
 [1.19869472 0.49527527 0.         0.        ]
 [1.19338881 0.49563984 0.         0.        ]
 [1.19827176 0.49862446 0.         0.        ]
 [1.20921918 0.49076261 0.         0.        ]
 [1.19170242 0.49121204 0.         0.        ]
 [1.20663786 0.49187918 0.         0.        ]
 [1.19134428 0.49247348 0.         0.        ]
 [1.19423396 0.49571277 0.         0.        ]]
[[1.19421775 0.5034489  0.         0.        ]
 [1.2092529

  0%|                                                   | 0/125 [00:00<?, ?it/s]/Users/b6062805/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:2559: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real
/Users/b6062805/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:2560: RuntimeWarning: overflow encountered in reduce
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))
  1%|▎                                          | 1/125 [00:00<01:01,  2.03it/s]/usr/local/anaconda3/lib/python3.8/site-packages/autokoopman/estimator/koopman.py:83: RuntimeWarning: overflow encountered in matmul
  return np.real(self._A @ obs.T).flatten()[: len(x)]
/usr/local/anaconda3/lib/python3.8/site-packages/autokoopman/estimator/koopman.py:83: RuntimeWarning: invalid value encountered in matmul
  return np.real(self._A @ obs.T).flatten()[: len(x)]
/usr/local/anaconda3/lib/python3.8/site-packages/scipy/interpolate/_interpolate.py:641: RuntimeWarning: overflow encount

[[1.20946552 0.49267146 0.         0.        ]
 [1.19628034 0.50496835 0.         0.        ]
 [1.20657264 0.50035609 0.         0.        ]
 [1.20323923 0.50363432 0.         0.        ]
 [1.19636635 0.49182558 0.         0.        ]
 [1.20867689 0.50971769 0.         0.        ]
 [1.20157559 0.50053026 0.         0.        ]
 [1.1992088  0.50876527 0.         0.        ]
 [1.20941269 0.50256441 0.         0.        ]
 [1.19489117 0.49499647 0.         0.        ]]
[[1.20319574 0.50287188 0.         0.        ]
 [1.19008078 0.50768314 0.         0.        ]
 [1.20339997 0.50357676 0.         0.        ]
 [1.19699348 0.50869836 0.         0.        ]
 [1.20591006 0.50257132 0.         0.        ]
 [1.19451173 0.49222194 0.         0.        ]
 [1.20060594 0.49997052 0.         0.        ]
 [1.19653977 0.50314711 0.         0.        ]
 [1.19035837 0.49464507 0.         0.        ]
 [1.20409856 0.49873489 0.         0.        ]]
[[1.19341808 0.50107244 0.         0.        ]
 [1.1921261

  0%|                                                   | 0/250 [00:00<?, ?it/s]/Users/b6062805/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:2559: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real
/Users/b6062805/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:2560: RuntimeWarning: overflow encountered in reduce
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))
  0%|▏                                          | 1/250 [00:00<02:06,  1.97it/s]/usr/local/anaconda3/lib/python3.8/site-packages/autokoopman/estimator/koopman.py:83: RuntimeWarning: overflow encountered in matmul
  return np.real(self._A @ obs.T).flatten()[: len(x)]
/usr/local/anaconda3/lib/python3.8/site-packages/autokoopman/estimator/koopman.py:83: RuntimeWarning: invalid value encountered in matmul
  return np.real(self._A @ obs.T).flatten()[: len(x)]
/usr/local/anaconda3/lib/python3.8/site-packages/scipy/interpolate/_interpolate.py:641: RuntimeWarning: overflow encount